In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def getPath():
    
    # add your path to this list
    paths = ['/Users/nazan/Desktop/SF Housing/Data/']

    for path in paths:
        if os.path.exists(path):
            mainPath = path
            break
        else:
            raise Exception('Your path not found. Add it to the list of paths')
    return mainPath
    
def loadApplicantData():
    path = getPath()
    applicantDf = pd.read_excel(path+'Master BMR Project List - IH 2018114.xlsx')
    return applicantDf

def loadApplicantData1():
    path = getPath()
    applicantDf1 = pd.read_excel(path+'MasterBMR_Walkscore.csv')
    return applicantDf1

def loadApplicantData2():
    path = getPath()
    applicantDf2 = pd.read_excel(path+'alltransit_data_blkgrps_SF.csv')
    return applicantDf2

import requests
import re

def convert_match(match):
    # returns the score value, returns -1 if null
    if match:
        score = match.group(1)
        if score == "null":
            score = -1
        else:
            score = float(score)
    else:
        score = None
    return score
    
def parse_output(text):
    walk = convert_match(re.search(r"\"walkscore\": ([\w\d]+)", text))
    transit = convert_match(re.search(r"\"transit\" : {\"score\": ([\w\d]*)", text))
    bike = convert_match(re.search(r"\"bike\" : {\"score\": ([\w\d]*)", text))
    return (walk, transit, bike)

def get_scores(address, lat, lon, key='13ed724b1cb7f6f61ed46bea1629502c'):
    address = address.replace(' ', '%20')
    lookup = "http://api.walkscore.com/score?format=json&address=" + address + \
    "&lat=" + str(lat) + "&lon=" + str(lon) + "&transit=1&bike=1&wsapikey=" + key
    req = requests.get(lookup)
    return parse_output(req.text)

In [3]:
def final_table(resi_proj,output_path):
    master_BMR = loadApplicantData() # the BMR table
    proj = pd.read_excel(resi_proj)
    mod_proj = proj[proj['Project ID'].isin(master_BMR['Project ID'])]
    mod_proj.index = range(len(mod_proj))

    walkscore = []
    transitscore = []
    bikescore = []
    
    for idx, row in mod_proj.iterrows():
        project_id = row['Project ID']
        address = str(row['Street Number'])+' '+row['Street Name']+', San Francisco, CA'
        lat =row['Latitude']
        lon =row['Longitude']
        scores = get_scores(address, lat = lat, lon= lon) # may need to get lat and lon as well
        walkscore.append(scores[0])
        transitscore.append(scores[1])
        bikescore.append(scores[2])

    mod_proj['walkscore'] = walkscore
    mod_proj['transitscore'] = transitscore
    mod_proj['bikescore'] = bikescore
    
    mod_proj.to_csv(output_path)
    
    #return mod_proj

In [4]:
final_table("/Users/nazan/Desktop/SF Housing/Data/Residential Projects with Inclusionary Requirements Updated FILTERED (proper latitude).xlsx",
               "/Users/nazan/Desktop/SF Housing/Data/MasterBMR_Walkscore.csv")

C:\Users\nazan\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\nazan\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\nazan\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi

In [ ]:
def final_file(CNT,walkscore):
    walkscore = loadApplicantData1() # the MasterBMR_Walkscore table
    proj = pd.read_excel('alltransit_data_blkgrps_SF.csv')
    bmr_proj = proj[proj['blkgrp'].isin(walkscore['GEOID'])]
    bmr_proj.index = range(len(bmr_proj))

In [ ]:
merged = bmr_proj.merge(walkscore, how = 'right' on='GEOID')
merged.to_csv("MasterBMR_Final.csv", index=False)